## **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the other exploratory data analysis notebooks, there are visualizations of the SpaceX launch dataset using the `matplotlib` and `seaborn` libraries. Such libraries are useful to discover preliminary correlations between the variables in the dataset.

In this notebook it is presented how to perform interactive visual analytics using the library `Folium`.


## Objectives


*   Marking all launch sites on a map.
*   Marking the success/failed launches for each site on the map.
*   Calculating the distances between a launch site and its proximities.

This notebook is an example of exploratory analysis to find geographical patterns in launch sites.

### Libraries

In [1]:
# Create an environment with Python >=3.7 & <=3.11 for library compatibility
# conda create -n basicMachineLearning python=3.7
# conda install -n basicMachineLearning ipykernel --update-deps --force-reinstall

# Libraries required.
# pip install pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 
# scikit-learn==0.20.1 requests beautifulsoup4 ipython-sql sqlalchemy==1.3.9
# folium

In [2]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

import pandas as pd

import requests
import io

Adding each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(URL)
text = io.StringIO(resp.text)

spacex_df = pd.read_csv(text)
print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


Coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


The above coordinates are just plain numbers and don't bring any insights about the location of the launch sites. It is more useful to pin the locations on a map to be able to inspect them visually.


Creating a folium `Map` object, with an initial center location in the NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

`folium.Circle` adds a highlighted circle with a text label on a specific coordinate.


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

There is a small orange circle near the city of Houston. The map is interactive, so it allows to zoom-in and see a larger circle.


Adding a circle for each launch site in the data frame


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [13]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map2 = folium.Map(location=nasa_coordinate, zoom_start=10)

# Initiating the map
site_map2 = folium.Map(location=nasa_coordinate, zoom_start=5)
site_map2
coordinates = [] 
site_names = [] 
# Iterating over the launch sites
for iLaunch in range(launch_sites_df.shape[0]):
    coordinate = launch_sites_df[['Lat','Long']].iloc[iLaunch].tolist()
    site_name = launch_sites_df[['Launch Site']].iloc[iLaunch].tolist()
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site_name[0]))
    # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        coordinate,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name[0],
            )
        )
    site_map2.add_child(circle)
    site_map2.add_child(marker)
site_map2


All launch sites are close to the ecuator line since it can take advantage of the Earth's rotational speed. Also, they are close to the ocean/ sea so the rocket's paths are not over populated areas.


### Marking success / fail launches on the map


In [20]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Creating markers for all launch records.
Using a green marker for a successful launch `(class=1)`, and a red marker for a failed launch `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [15]:
marker_cluster = MarkerCluster()


Creating a new column in `spacex_df` called `marker_color` to store the marker colors based on the `class` value


In [19]:
# Applying a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Function to determine marker color
def determine_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function
spacex_df['marker_color'] = spacex_df.apply(determine_color, axis=1)

For each launch result in `spacex_df` data frame, adding a `folium.Marker` to `marker_cluster`


In [26]:
del site_map3

In [27]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map3 = folium.Map(location=nasa_coordinate, zoom_start=10)
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map3.add_child(marker_cluster)

site_names = [] 
# Iterate over the DataFrame
for index, row in spacex_df.iterrows():
    # Create a Marker for each launch site
    marker = folium.Marker(
        [row['Lat'], row['Long']],
        # Customize the marker icon
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    )
    # Add marker to the cluster
    marker_cluster.add_child(marker)

    marker_tag = folium.map.Marker(
        [row['Lat'], row['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
            )
        )
    site_map3.add_child(marker_tag)

# Add MarkerCluster to the map
site_map3.add_child(marker_cluster)

# Display the map
site_map3

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, it is easy to identify which launch sites have a high success rates.


### Calculating the distance between a launch site and proximities

To explore and analyze the proximities of launch sites it is possible to add a `MousePosition` object to get the coordinates of the cursor over a point on the map.


In [28]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [29]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [42]:
coordinates = [[28.5631, -80.5738],[28.5627, -80.5777]]
distance_coastline = calculate_distance(coordinates[0][0], coordinates[0][1], coordinates[1][0], coordinates[1][1])

In [43]:
# Creating and adding a folium.Marker on the selected closest coastline point on the map
# Displaying the distance between coastline point and launch site using the icon property 

distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )

Drawing a `PolyLine` between the launch site and the selected coastline point.


In [44]:
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)